# Constants

In [1]:
# ! az login --tenant 3ad0b905-34ab-4116-93d9-c1dcc2d35af6

In [2]:
import os
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
    sys.exit()
print("Environment variables have been loaded ;-)")

deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME_MINI"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Environment variables have been loaded ;-)
Project Connection String: <...hub01-grp;mmai-swc-hub01-prj01>


# Create AI Foundry Project Client

In [3]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), 
    conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai-swc-hub01-grp',
 'project_name': 'mmai-swc-hub01-prj01'}

# Just for testing: List current AI Foundry Agents

In [4]:
agents = project_client.agents.list_agents()
agents

{'object': 'list', 'data': [{'id': 'asst_mxbc98kziO1XIJKOUbWh8R14', 'object': 'assistant', 'created_at': 1745603693, 'name': 'Sage (for AI Search, offering deep knowledge retrieval)', 'description': None, 'model': 'gpt-4o', 'instructions': '', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_lPKlxViYS4as2edgZlwsGEPz', 'object': 'assistant', 'created_at': 1745603676, 'name': 'Navigator (for Trip Advisor, guiding users through travel insights)', 'description': None, 'model': 'gpt-4o', 'instructions': '', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_uGwQrkuV7wIqy1X7fPVZbEU5', 'object': 'assistant', 'created_at': 1745603672, 'name': 'Insight agent (for Bing-powered search intelligence)', 'description': None, 'model': 'gpt-4o-mini', 'instructions': '', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {},

In [5]:
# easier to read ;-)

[f"{a.id}: {a.name}" for a in agents.data]

['asst_mxbc98kziO1XIJKOUbWh8R14: Sage (for AI Search, offering deep knowledge retrieval)',
 'asst_lPKlxViYS4as2edgZlwsGEPz: Navigator (for Trip Advisor, guiding users through travel insights)',
 'asst_uGwQrkuV7wIqy1X7fPVZbEU5: Insight agent (for Bing-powered search intelligence)',
 'asst_P4wLkpQc1VV5tjb3XpLgNMQg: Oracle (for OpenAI-powered reasoning and content generation',
 'asst_eAvAKNCADL9iW0xgtMMPgWTv: Scout (for broad web discovery and intelligence gathering)',
 'asst_QpVzw6SaAtyrue9LK3eidwww: Pulse (for monitoring trends and real-time data analysis)',
 'asst_gRXFM2Hb97USgxrD8Yb0kjZf: Beacon (for AI-driven recommendations and guidance)']

# Create the CodeInterpreterTool

In [6]:
from azure.ai.projects.models import CodeInterpreterTool

code_interpreter = CodeInterpreterTool()
print(f"Code interpreter definitions: {code_interpreter.definitions}")
print(f"Code interpreter resources: {code_interpreter.resources}")

Code interpreter definitions: [{'type': 'code_interpreter'}]
Code interpreter resources: {}


# Create AI Foundry Agent with its CodeInterpreterTool for AI Agents

In [7]:
# ...then, create the AI Foundry Agent, attaching CodeInterpreterTool object
ai_agent = project_client.agents.create_agent(
    model=deployment_name,
    name="my-agent001",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)

ai_agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_kztRjkdTvWGlp1DYMWBwGQQd', 'object': 'assistant', 'created_at': 1745604232, 'name': 'my-agent001', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>

# Just for testing: create a new agent and delete it

In [8]:
# Create a second agent
ai_agent_02 = project_client.agents.create_agent(
    model=deployment_name,
    name="my-agent002 TO BE DELETED",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)

# List all agents
print(f"All agents, including the useless one:")
display([f"{a.id}: {a.name}" for a in project_client.agents.list_agents()['data']])

project_client.agents.delete_agent(ai_agent_02.id)

# List all agents
print(f"\nAll agents, after deleting the useless one:")
display([f"{a.id}: {a.name}" for a in project_client.agents.list_agents()['data']])

All agents, including the useless one:


['asst_r1EqIGRYSxZ0IvbTTuozxOh7: my-agent002 TO BE DELETED',
 'asst_kztRjkdTvWGlp1DYMWBwGQQd: my-agent001',
 'asst_mxbc98kziO1XIJKOUbWh8R14: Sage (for AI Search, offering deep knowledge retrieval)',
 'asst_lPKlxViYS4as2edgZlwsGEPz: Navigator (for Trip Advisor, guiding users through travel insights)',
 'asst_uGwQrkuV7wIqy1X7fPVZbEU5: Insight agent (for Bing-powered search intelligence)',
 'asst_P4wLkpQc1VV5tjb3XpLgNMQg: Oracle (for OpenAI-powered reasoning and content generation',
 'asst_eAvAKNCADL9iW0xgtMMPgWTv: Scout (for broad web discovery and intelligence gathering)',
 'asst_QpVzw6SaAtyrue9LK3eidwww: Pulse (for monitoring trends and real-time data analysis)',
 'asst_gRXFM2Hb97USgxrD8Yb0kjZf: Beacon (for AI-driven recommendations and guidance)']


All agents, after deleting the useless one:


['asst_kztRjkdTvWGlp1DYMWBwGQQd: my-agent001',
 'asst_mxbc98kziO1XIJKOUbWh8R14: Sage (for AI Search, offering deep knowledge retrieval)',
 'asst_lPKlxViYS4as2edgZlwsGEPz: Navigator (for Trip Advisor, guiding users through travel insights)',
 'asst_uGwQrkuV7wIqy1X7fPVZbEU5: Insight agent (for Bing-powered search intelligence)',
 'asst_P4wLkpQc1VV5tjb3XpLgNMQg: Oracle (for OpenAI-powered reasoning and content generation',
 'asst_eAvAKNCADL9iW0xgtMMPgWTv: Scout (for broad web discovery and intelligence gathering)',
 'asst_QpVzw6SaAtyrue9LK3eidwww: Pulse (for monitoring trends and real-time data analysis)',
 'asst_gRXFM2Hb97USgxrD8Yb0kjZf: Beacon (for AI-driven recommendations and guidance)']

# Create the thread and attach a new message to it

In [9]:
# Create a thread
thread = project_client.agents.create_thread()

print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    """,
)
print(f"Messages: {project_client.agents.list_messages(thread_id=thread.id)}")

Created thread: {'id': 'thread_2QztOeDdGFj17fLJvU1SpJco', 'object': 'thread', 'created_at': 1745604235, 'metadata': {}, 'tool_resources': {}}

Messages: {'object': 'list', 'data': [{'id': 'msg_9ycImSmJfde5XhNKMCcCyAMP', 'object': 'thread.message', 'created_at': 1745604235, 'assistant_id': None, 'thread_id': 'thread_2QztOeDdGFj17fLJvU1SpJco', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': '\n    Could you please create a bar chart for the operating profit using \n    the following data and provide the file to me? \n    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, \n    Company D: $1.8 million\n    ', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_9ycImSmJfde5XhNKMCcCyAMP', 'last_id': 'msg_9ycImSmJfde5XhNKMCcCyAMP', 'has_more': False}


# Run the agent syncrhonously

In [10]:
%%time

# Run the agent
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, assistant_id=ai_agent.id)

print(f"Run status: {run.status}.\n\nRun details: {run}")

Run status: RunStatus.COMPLETED.

Run details: {'id': 'run_dDiaMt8ThRNeZwHXIYPbnWSM', 'object': 'thread.run', 'created_at': 1745604237, 'assistant_id': 'asst_kztRjkdTvWGlp1DYMWBwGQQd', 'thread_id': 'thread_2QztOeDdGFj17fLJvU1SpJco', 'status': 'completed', 'started_at': 1745604237, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1745604244, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 622, 'completion_tokens': 212, 'total_tokens': 834, 'prompt_token_details': {'cached_tokens': 0}}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: total: 78.1 ms
Wall time: 9.5 s


# Fetch messages from the thread after the agent run execution

In [11]:
display(project_client.agents.list_messages(thread_id=thread.id)["data"][0])
print("\n")
display(project_client.agents.list_messages(thread_id=thread.id)["data"][1])

{'id': 'msg_N1oxN1vmSeqNwr2TPXotKjbt', 'object': 'thread.message', 'created_at': 1745604243, 'assistant_id': 'asst_kztRjkdTvWGlp1DYMWBwGQQd', 'thread_id': 'thread_2QztOeDdGFj17fLJvU1SpJco', 'run_id': 'run_dDiaMt8ThRNeZwHXIYPbnWSM', 'role': 'assistant', 'content': [{'type': 'image_file', 'image_file': {'file_id': 'assistant-Jn9ebwfDxdbtBcF2jHhjKS'}}, {'type': 'text', 'text': {'value': 'I have created the bar chart for the operating profits of the companies. You can download it using the link below:\n\n[Download Operating Profit Chart](sandbox:/mnt/data/operating_profit_chart.png)', 'annotations': [{'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_chart.png', 'start_index': 150, 'end_index': 194, 'file_path': {'file_id': 'assistant-Xtxficwj29iuWfhJ5eaTms'}}]}}], 'attachments': [{'file_id': 'assistant-Xtxficwj29iuWfhJ5eaTms', 'tools': [{'type': 'code_interpreter'}]}], 'metadata': {}}

{'id': 'msg_9ycImSmJfde5XhNKMCcCyAMP', 'object': 'thread.message', 'created_at': 1745604235, 'assistant_id': None, 'thread_id': 'thread_2QztOeDdGFj17fLJvU1SpJco', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': '\n    Could you please create a bar chart for the operating profit using \n    the following data and provide the file to me? \n    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, \n    Company D: $1.8 million\n    ', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [12]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: 
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    

===== MESSAGE 2 =====

CONTENT 1 (MessageImageFileContent) --> image_file id: assistant-Jn9ebwfDxdbtBcF2jHhjKS

CONTENT 2 (MessageTextContent) --> Text: I have created the bar chart for the operating profits of the companies. You can download it using the link below:

[Download Operating Profit Chart](sandbox:/mnt/data/operating_profit_chart.png)
>>> Annotation in MessageTextContent 2 of message 2: sandbox:/mnt/data/operating_profit_chart.png



# Retrieve and download eventual annotations

In [13]:
# messages.file_path_annotations[0].text.split('/')[-1]
print (f"Nr. of file path annotations: {len(messages.file_path_annotations)}\n")

i=0
for file_path_annotation in messages.file_path_annotations:
    i += 1
    print(f"{i} - File annotation paths: {file_path_annotation}")
    file_name = file_path_annotation.text.split('/')[-1]
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annoation {i} saved as file to: {os.getcwd()}\\{file_name}")

Nr. of file path annotations: 1

1 - File annotation paths: {'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_chart.png', 'start_index': 150, 'end_index': 194, 'file_path': {'file_id': 'assistant-Xtxficwj29iuWfhJ5eaTms'}}
File annoation 1 saved as file to: E:\Users\mauromi\source\git_repos\aaas\operating_profit_chart.png


# Retrieve and download eventual images

In [14]:
print (f"Nr. of image contents: {len(messages.image_contents)}\n")

i=0
# Generate an image file for the bar chart
for image_content in messages.image_contents:
    i += 1
    print(f"{i} - Image content: {image_content}")
    file_name = f"{image_content.image_file.file_id}_image_content.png"
    project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}\\{file_name}")

Nr. of image contents: 1

1 - Image content: {'type': 'image_file', 'image_file': {'file_id': 'assistant-Jn9ebwfDxdbtBcF2jHhjKS'}}
Image content 1 file to: E:\Users\mauromi\source\git_repos\aaas\assistant-Jn9ebwfDxdbtBcF2jHhjKS_image_content.png


# Teardown for *all** agents and messages

- Agent 2: asst_lPKlxViYS4as2edgZlwsGEPz (Navigator (for Trip Advisor, guiding users through travel insights)) created at 2025-04-25 19:54:36...
- Agent 3: asst_P4wLkpQc1VV5tjb3XpLgNMQg (Oracle (for OpenAI-powered reasoning and content generation) created at 2025-04-25 19:54:18...
- Agent 4: asst_QpVzw6SaAtyrue9LK3eidwww (Pulse (for monitoring trends and real-time data analysis)) created at 2025-04-25 19:54:05...
- Agent 5: asst_mxbc98kziO1XIJKOUbWh8R14 (Sage (for AI Search, offering deep knowledge retrieval)) created at 2025-04-25 19:54:53...
- Agent 6: asst_eAvAKNCADL9iW0xgtMMPgWTv (Scout (for broad web discovery and intelligence gathering)) created at 2025-04-25 19:54:09...
- Agent 7: asst_uGwQrkuV7wIqy1X7fPVZbEU5 (Insight agent (for Bing-powered search intelligence)) created at 2025-04-25 19:54:32...
- Agent 8: asst_gRXFM2Hb97USgxrD8Yb0kjZf (Beacon (for AI-driven recommendations and guidance)) created at 2025-04-25 19:53:51...

In [51]:
def list_agents(pc: AIProjectClient, delete:bool=False):
    agents = pc.agents.list_agents(limit=100)['data']

    i=0
    while len(agents) > 0:
        for agent in agents:
            i=i+1
            if delete:
                print(f"Deleting agent {i}: {agent.id} ({agent.name}) created at {datetime.fromtimestamp(agent['created_at']).strftime('%Y-%m-%d %H:%M:%S')}...")
                # client.beta.assistants.delete(assistant.id)
                pc.agents.delete_agent(id = agent.id)
            else:
                print(f"Agent {i}: {agent.id} ({agent.name}) created at {datetime.fromtimestamp(agent['created_at']).strftime('%Y-%m-%d %H:%M:%S')}...")

            agents.remove(agent)
            #agents = project_client.agents.list_agents(limit=100)['data']
    
    print (f"\n{i} agents have been successfully processed.")


list_agents(project_client, delete=True)


0 agents have been successfully processed.


In [24]:
from datetime import datetime
agents = list_agents.agents.list_agents(limit=100)['data']

i=0
while len(agents) > 0:
    for agent in agents:
        i=i+1
        print(agent.created_at)
        print(f"Deleting agent {i}: {agent.id} ({agent.name}) created at {datetime.fromtimestamp(agent['created_at']).strftime('%Y-%m-%d %H:%M:%S')}...")
        # client.beta.assistants.delete(assistant.id)
    agents = project_client.agents.list_agents(limit=100)['data']

print (f"\n{i} agents have been successfully deleted.")

2025-04-25 18:03:52+00:00
Deleting agent 1: asst_kztRjkdTvWGlp1DYMWBwGQQd (my-agent001) created at 2025-04-25 20:03:52...
2025-04-25 17:54:53+00:00
Deleting agent 2: asst_mxbc98kziO1XIJKOUbWh8R14 (Sage (for AI Search, offering deep knowledge retrieval)) created at 2025-04-25 19:54:53...
2025-04-25 17:54:36+00:00
Deleting agent 3: asst_lPKlxViYS4as2edgZlwsGEPz (Navigator (for Trip Advisor, guiding users through travel insights)) created at 2025-04-25 19:54:36...
2025-04-25 17:54:32+00:00
Deleting agent 4: asst_uGwQrkuV7wIqy1X7fPVZbEU5 (Insight agent (for Bing-powered search intelligence)) created at 2025-04-25 19:54:32...
2025-04-25 17:54:18+00:00
Deleting agent 5: asst_P4wLkpQc1VV5tjb3XpLgNMQg (Oracle (for OpenAI-powered reasoning and content generation) created at 2025-04-25 19:54:18...
2025-04-25 17:54:09+00:00
Deleting agent 6: asst_eAvAKNCADL9iW0xgtMMPgWTv (Scout (for broad web discovery and intelligence gathering)) created at 2025-04-25 19:54:09...
2025-04-25 17:54:05+00:00
Deleti

KeyboardInterrupt: 

In [21]:
print (f"Nr. of existing agents that will be deleted: {len(project_client.agents.list_agents()['data'])}\n")

i = 0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"{i} - Agent {pca.name} ({pca.id}) is not deleted.")

Nr. of existing agents that will be deleted: 9

1 - Agent my-agent001 (asst_nawLaEsu4lEr5odgku0qcyC4) is not deleted.
2 - Agent Beacon (for AI-driven recommendations and guidance) (asst_N1lmbzfeWyoIqydIk1jsU8Ic) is not deleted.
3 - Agent Pulse (for monitoring trends and real-time data analysis) (asst_Wq1Ofun2LdK04BZcK526Fd13) is not deleted.
4 - Agent Scout (for broad web discovery and intelligence gathering) (asst_Fm51pQ0afAJC7rWyzqefyK79) is not deleted.
5 - Agent Oracle (for OpenAI-powered reasoning and content generation) (asst_ZxEMkcCQS0bDOQQQCrveRdNz) is not deleted.
6 - Agent Synth (for Fabric, focused on data synthesis and visualization) (asst_U9h4ux1iTdue8dF9yArV6Aha) is not deleted.
7 - Agent Navigator (for Trip Advisor, guiding users through travel insights) (asst_3MkOnTUgjcJozbjU5CCio5dk) is not deleted.
8 - Agent Sage (for AI Search, offering deep knowledge retrieval) (asst_HGsDJnktHqhrWXSNp4KH8Fd9) is not deleted.
9 - Agent Insight agent (for Bing-powered search intellige

# HIC SUNT LEONES